In [1]:
import pandas as pd
import sqlite3
import concurrent.futures
import time
import re

from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer

In [9]:
text=f"""Account Director of Integrated Account Department PT Huawei Tech Investment, Mukti Ali, akan diperiksa sebagai terdakwa kasus dugaan korupsi proyek BTS 4G Kominfo. Pemeriksaan akan dilakukan pada Senin, 23 Oktober 2023."Agenda sidang kita berikutnya di hari Senin, tanggal 23 Oktober 2023. Setelah saksi dilanjutkan pemeriksaan terdakwa ya," kata ketua hakim Dennie Arsan Fatrika dalam persidangan di PN Tipikor Jakarta, Jalan Bungur Besar Raya, Jakarta Pusat, Jumat (20/10/2023).Dennie mengatakan Mukti Ali juga akan diperiksa sebagai saksi mahkota terlebih dahulu. Kemudian dilanjutkan dengan pemeriksaan sebagai terdakwa."Demikian ya, jadi untuk persidangan berikutnya ditunda sampai dengan hari Senin, tanggal 23 Oktober 2023 ya dengan agenda untuk pemeriksaan saksi, masing-masing terdakwa sebagai saksi untuk perkara terdakwa lainnya dan dilanjutkan pemeriksaan terdakwa," ujarnya.Komisaris PT Solitech Media Sinergy Irwan Hermawan didakwa melakukan korupsi dalam proyek pengadaan base transceiver station (BTS) 4G dan infrastruktur pendukung paket 1, 2, 3, 4, dan 5 Bakti Kementerian Komunikasi dan Informatika tahun 2020-2022. Irwan didakwa merugikan negara Rp 8 triliun.Irwan diadili bersama Account Director of Integrated Account Department PT Huawei Tech Investment Mukti Ali dan Direktur Utama PT Mora Telematika Indonesia Galumbang Menak Simanjuntak. Mereka didakwa dalam berkas terpisah.Dalam dakwaan yang dibacakan jaksa dalam sidang di Pengadilan Tipikor Jakarta Pusat, Selasa (4/7/2023), Irwan beserta Tenaga Ahli Human Development Universitas Indonesia Tahun 2020 Yohan Suryanto, Direktur Utama Bakti Kominfo Anang Achmad Latif dan Galumbang serta Mukti melakukan pertemuan-pertemuan dengan calon kontraktor dan subkontraktor dalam rangka menentukan pelaksana pekerjaan. Pertemuan itu mengatur persyaratan pemilihan penyedia.Kemudian Irwan menentukan pemenang penyedia, yakni Konsorsium Fiber Home PT Telkominfra dan PT Multi Trans Data (PT MTD) untuk Paket 1, 2, lalu Konsorsium PT Lintas Arta, PT Huawei, dan PT Surya Energy Indotama (SEI) untuk Paket 3, serta Konsorsium PT Infra Struktur Bisnis Sejahtera (IBS) dan PT ZTE Indonesia Paket 4, 5."Anang Achmad Latif memerintahkan Ferandi Mirza untuk membentuk tim bayangan, yang terdiri dari Gandi, Avrinson, Maryulis, Edy untuk memastikan Pokja melaksanakan kriteria yang sudah ditentukan Anang Achmad Latif bersama-sama dengan Terdakwa Irwan Hermawan dan Galumbang Menak Simanjuntak," lanjut jaksa.Simak Video 'Sesal Johnny G Plate Proyek BTS Tak Selesai, Tapi Tak Merasa Salah':[Gambas:Video 20detik]Baca halaman selanjutnya>>Halaman 1 2 Selanjutnya bts kominfo kasus bts kominfo kasus korupsi bts kominfo korupsi bts kominfo hukum"""
content_cleaned = re.sub(r'ADVERTISEMENT', '', text)
content_cleaned = re.sub(r'Detik News', '', content_cleaned)
content_cleaned = re.sub(r'CNN News', '', content_cleaned)
content_cleaned = re.sub(r'KOMPAS.com', '', content_cleaned)
content_cleaned = re.sub(r'Kompas News', '', content_cleaned)
content_cleaned = re.sub(r'Gambas', '', content_cleaned)
content_cleaned = re.sub(r'20detik', '', content_cleaned)
content_cleaned = re.sub(r'berikutnya', '', content_cleaned)
content_cleaned = re.sub(r'halaman', '', content_cleaned)
content_cleaned = re.sub(r'detikcom', '', content_cleaned)
content_cleaned = re.sub(r'Halaman', '', content_cleaned)




In [10]:
print(content_cleaned)

Account Director of Integrated Account Department PT Huawei Tech Investment, Mukti Ali, akan diperiksa sebagai terdakwa kasus dugaan korupsi proyek BTS 4G Kominfo. Pemeriksaan akan dilakukan pada Senin, 23 Oktober 2023."Agenda sidang kita  di hari Senin, tanggal 23 Oktober 2023. Setelah saksi dilanjutkan pemeriksaan terdakwa ya," kata ketua hakim Dennie Arsan Fatrika dalam persidangan di PN Tipikor Jakarta, Jalan Bungur Besar Raya, Jakarta Pusat, Jumat (20/10/2023).Dennie mengatakan Mukti Ali juga akan diperiksa sebagai saksi mahkota terlebih dahulu. Kemudian dilanjutkan dengan pemeriksaan sebagai terdakwa."Demikian ya, jadi untuk persidangan  ditunda sampai dengan hari Senin, tanggal 23 Oktober 2023 ya dengan agenda untuk pemeriksaan saksi, masing-masing terdakwa sebagai saksi untuk perkara terdakwa lainnya dan dilanjutkan pemeriksaan terdakwa," ujarnya.Komisaris PT Solitech Media Sinergy Irwan Hermawan didakwa melakukan korupsi dalam proyek pengadaan base transceiver station (BTS) 4G